In [1]:
5

5

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt

In [6]:
data_dir = "D:\MACHINE LEARNING\Indus_Internship2\data_augm"  # Directory containing train and valid folders
train_dir = os.path.join(data_dir, "train50")
valid_dir = os.path.join(data_dir, "valid")

# Image size used by ResNet
img_size = (128, 128)
batch_size = 16

train_ds = image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

valid_ds = image_dataset_from_directory(
    valid_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

class_names = train_ds.class_names
print("Total disease classes are:", len(class_names))

Found 35141 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.
Total disease classes are: 38


In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y)).cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
valid_ds = valid_ds.map(lambda x, y: (preprocess_input(x), y)).cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

base_model = ResNet50(include_top=False, input_shape=(128, 128, 3), weights='imagenet')
base_model.trainable = False  # Freeze for transfer learning

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [4]:
from tensorflow.keras.models import load_model

model = load_model('updated_resnet80.h5')

In [ ]:
# # Evaluate
# loss, acc = model.evaluate(valid_ds)
# print(f"Validation Accuracy: {acc*100:.2f}%")

# Predict
def predict_image(path):
    img = tf.keras.utils.load_img(path, target_size=(128, 128))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions)]
    confidence = np.max(predictions)

    return predicted_class, confidence

In [14]:
import os
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input

def predict_images_from_folder(folder_path, model, class_names):
    for fname in os.listdir(folder_path):
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            path = os.path.join(folder_path, fname)

            # Load and preprocess image
            img = load_img(path, target_size=(128, 128))
            img_array = img_to_array(img)
            img_array = preprocess_input(img_array)
            img_array = np.expand_dims(img_array, axis=0)

            # Predict
            preds = model.predict(img_array)
            pred_label = class_names[np.argmax(preds)]
            confidence = np.max(preds)

            print(f"{fname} → {pred_label} ({confidence:.2f})")

test_dir = "data_augm/itest"
predict_images_from_folder(test_dir, model, class_names)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
AppleCedarRust1.JPG → Apple___Cedar_apple_rust (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
AppleCedarRust2.JPG → Cherry_(including_sour)___healthy (0.49)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
AppleCedarRust3.JPG → Apple___Cedar_apple_rust (0.83)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
AppleCedarRust4.JPG → Apple___Cedar_apple_rust (0.97)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
AppleScab1.JPG → Apple___Cedar_apple_rust (0.97)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
AppleScab2.JPG → Peach___Bacterial_spot (0.43)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
AppleScab3.JPG → Apple___Apple_scab (0.53)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
CornCommonRust1.JPG → Corn_(maize)___Common_rust_ (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
CornCommonRust2.JPG → Corn_(maize)___Common_rust_ (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
CornCommonRust3.JPG → Corn_(maize)___Common_rust_ (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
PotatoEarlyBlight1.JPG → Potato___Early_blight (1

In [15]:
model2 = load_model('updated_resnet100.h5')

In [16]:
import os
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input

def predict_images_from_folder(folder_path, model, class_names):
    for fname in os.listdir(folder_path):
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            path = os.path.join(folder_path, fname)

            # Load and preprocess image
            img = load_img(path, target_size=(128, 128))
            img_array = img_to_array(img)
            img_array = preprocess_input(img_array)
            img_array = np.expand_dims(img_array, axis=0)

            # Predict
            preds = model.predict(img_array)
            pred_label = class_names[np.argmax(preds)]
            confidence = np.max(preds)

            print(f"{fname} → {pred_label} ({confidence:.2f})")

test_dir = "data_augm/itest"
predict_images_from_folder(test_dir, model2, class_names)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 875ms/step
AppleCedarRust1.JPG → Apple___Cedar_apple_rust (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
AppleCedarRust2.JPG → Cherry_(including_sour)___healthy (0.92)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
AppleCedarRust3.JPG → Apple___Cedar_apple_rust (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
AppleCedarRust4.JPG → Apple___healthy (0.65)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
AppleScab1.JPG → Apple___Apple_scab (0.82)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
AppleScab2.JPG → Peach___Bacterial_spot (0.85)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
AppleScab3.JPG → Apple___Apple_scab (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
CornCommonRust1.JPG → Corn_(maize)___Common_rust_ (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
CornCommonRust2.JPG → Corn_(maize)___Common_rust_ (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
CornCommonRust3.JPG → Corn_(maize)___Common_rust_ (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
PotatoEarlyBlight1.JPG → Potato___Early_blight (1.00)
1/1 ━━━

In [18]:
model3 = load_model('ishan_100__resnet.h5')

In [19]:
def predict_images_from_folder(folder_path, model, class_names):
    for fname in os.listdir(folder_path):
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            path = os.path.join(folder_path, fname)

            # Load and preprocess image
            img = load_img(path, target_size=(128, 128))
            img_array = img_to_array(img)
            img_array = preprocess_input(img_array)
            img_array = np.expand_dims(img_array, axis=0)

            # Predict
            preds = model.predict(img_array)
            pred_label = class_names[np.argmax(preds)]
            confidence = np.max(preds)

            print(f"{fname} → {pred_label} ({confidence:.2f})")

test_dir = "data_augm/itest"
predict_images_from_folder(test_dir, model3, class_names)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 898ms/step
AppleCedarRust1.JPG → Apple___Cedar_apple_rust (0.98)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
AppleCedarRust2.JPG → Apple___Cedar_apple_rust (0.55)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
AppleCedarRust3.JPG → Apple___Cedar_apple_rust (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
AppleCedarRust4.JPG → Apple___Cedar_apple_rust (0.99)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
AppleScab1.JPG → Apple___Apple_scab (0.53)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
AppleScab2.JPG → Peach___Bacterial_spot (0.91)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
AppleScab3.JPG → Apple___Black_rot (0.65)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
CornCommonRust1.JPG → Corn_(maize)___Common_rust_ (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
CornCommonRust2.JPG → Corn_(maize)___Common_rust_ (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
CornCommonRust3.JPG → Corn_(maize)___Common_rust_ (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
PotatoEarlyBlight1.JPG → Potato___Early_blight (0.99)
1/1 ━━━━